## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ushuaia,AR,-54.8000,-68.3000,46.06,93,75,25.32,moderate rain
1,1,Sao Joao Da Barra,BR,-21.6403,-41.0511,77.09,87,42,13.40,scattered clouds
2,2,Saskylakh,RU,71.9167,114.0833,-22.13,99,99,3.42,overcast clouds
3,3,Jamestown,US,42.0970,-79.2353,17.29,49,0,8.01,clear sky
4,4,Mataura,NZ,-46.1927,168.8643,56.55,95,100,1.83,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp_pref = float(input('What is the minimum temperature that you would like for your trip?'))
max_temp_pref = float(input('What is the maximum temperature that you would like for your trip?'))

What is the minimum temperature that you would like for your trip?65
What is the maximum temperature that you would like for your trip?85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
city_pref_df = city_data_df.loc[(city_data_df['Max Temp'] >= min_temp_pref) & (city_data_df['Max Temp'] <= max_temp_pref)]


In [5]:
# 4a. Determine if there are any empty rows.
city_pref_df.isnull().sum()

City_ID                0
City                   0
Country                5
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
city_pref_df = city_data_df.loc[(city_data_df['Max Temp'] >= min_temp_pref) & (city_data_df['Max Temp'] <= max_temp_pref)].dropna()
city_pref_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = city_pref_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Sao Joao Da Barra,BR,77.09,scattered clouds,-21.6403,-41.0511,
6,Hermanus,ZA,65.80,clear sky,-34.4187,19.2345,
7,Rikitea,PF,77.32,overcast clouds,-23.1203,-134.9692,
10,Waingapu,ID,78.28,overcast clouds,-9.6567,120.2641,
13,Busselton,AU,67.05,clear sky,-33.6500,115.3333,
15,Puerto Ayora,EC,80.56,few clouds,-0.7393,-90.3518,
17,Isangel,VU,77.11,heavy intensity rain,-19.5500,169.2667,
18,Cape Town,ZA,68.34,clear sky,-33.9258,18.4232,
19,Hobyo,SO,77.11,few clouds,5.3505,48.5268,
20,Port Alfred,ZA,70.25,scattered clouds,-33.5906,26.8910,


In [8]:
# 6a. Set parameters to search for hotels within 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    #add lat and lng to the params
    params['location'] = f'{lat}, {lng}'
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        print('No hotel found.....skipping')
        
        

No hotel found.....skipping
No hotel found.....skipping
No hotel found.....skipping
No hotel found.....skipping
No hotel found.....skipping
No hotel found.....skipping
No hotel found.....skipping
No hotel found.....skipping
No hotel found.....skipping
No hotel found.....skipping
No hotel found.....skipping
No hotel found.....skipping
No hotel found.....skipping
No hotel found.....skipping
No hotel found.....skipping
No hotel found.....skipping
No hotel found.....skipping
No hotel found.....skipping
No hotel found.....skipping
No hotel found.....skipping
No hotel found.....skipping
No hotel found.....skipping


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.replace('', np.nan, inplace=True)
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
hotel_df.isnull().sum()


City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [10]:
#preview dataframe
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Sao Joao Da Barra,BR,77.09,scattered clouds,-21.6403,-41.0511,Pousada Porto De Canoas
6,Hermanus,ZA,65.80,clear sky,-34.4187,19.2345,Misty Waves Boutique Hotel
7,Rikitea,PF,77.32,overcast clouds,-23.1203,-134.9692,People ThankYou
10,Waingapu,ID,78.28,overcast clouds,-9.6567,120.2641,Pondok Wisata Elim
13,Busselton,AU,67.05,clear sky,-33.6500,115.3333,Observatory Guest House
15,Puerto Ayora,EC,80.56,few clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
17,Isangel,VU,77.11,heavy intensity rain,-19.5500,169.2667,Tanna Lodge
18,Cape Town,ZA,68.34,clear sky,-33.9258,18.4232,Southern Sun Waterfront Cape Town
19,Hobyo,SO,77.11,few clouds,5.3505,48.5268,Halane Hotel and Restaurent
20,Port Alfred,ZA,70.25,scattered clouds,-33.5906,26.8910,The Halyards Hotel


In [11]:
# 8a. Create the output File (CSV)
path = 'WeatherPy_vacation.csv'
hotel_df.to_csv(path, index_label='City ID')


In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp}</dd>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations_df = hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer=gmaps.marker_layer(locations_df, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))